In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('/scratch/ab9738/dfdl_imputation/')
import numpy as np
import matplotlib.pyplot as plt
import SAUCIE
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

2023-06-10 16:17:43.798387: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-10 16:17:43.847486: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 16:17:44.519591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
y = np.transpose(np.load('../SERGIO/imputation_data/DS6_45.npy'))

In [4]:
x = np.transpose(np.load('../SERGIO/imputation_data/DS6_clean_counts.npy'))

In [5]:
x.shape

(2700, 1200)

In [6]:
y.shape

(2700, 1200)

In [7]:
len(np.where(y==0)[0])/(1200*2700)

0.9726978395061728

In [8]:
len(np.where(x==0)[0])/(1200*2700)

0.1744

In [9]:
np.sum(y.flatten()[np.where(x.flatten()==0)[0]])

4527

In [10]:
# tf.reset_default_graph()
# saucie = SAUCIE.SAUCIE(x.shape[1])
# loadtrain = SAUCIE.Loader(x, shuffle=True)
# saucie.train(loadtrain, steps=1000)

# loadeval = SAUCIE.Loader(x, shuffle=False)
# embedding = saucie.get_embedding(loadeval)
# number_of_clusters, clusters = saucie.get_clusters(loadeval)
# rec_x = saucie.get_reconstruction(loadeval)

tf.reset_default_graph()
saucie = SAUCIE.SAUCIE(y.shape[1])
loadtrain = SAUCIE.Loader(y, shuffle=True)
saucie.train(loadtrain, steps=1000)

loadeval = SAUCIE.Loader(y, shuffle=False)
# embedding = saucie.get_embedding(loadeval)
# number_of_clusters, clusters = saucie.get_clusters(loadeval)
rec_y = saucie.get_reconstruction(loadeval)

/scratch/ab9738/dfdl_imputation/SAUCIE/model.py:152: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h1 = tf.layers.dense(self.x, self.layers[0], activation=lrelu, name='encoder_0')
/scratch/ab9738/dfdl_imputation/SAUCIE/model.py:154: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h2 = tf.layers.dense(h1, self.layers[1], activation=tf.nn.sigmoid, name='encoder_1')
/scratch/ab9738/dfdl_imputation/SAUCIE/model.py:156: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h3 = tf.layers.dense(h2, self.layers[2], activation=lrelu, name='encoder_2')
/scratch/ab9738/dfdl_imputation/SAUCIE/model.py:158: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.embedded = t

In [11]:
np.save('yhat_saucie',rec_y)